In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7589,2024-05-28,Eua Nba,21:30,Dallas Mavericks,Minnesota Timberwolves,1.78,2.11,209.5,1.84,2.00,-2.5,2.04,2.45,tQzVri13,0.561798,0.473934,0.543478,0.500000,0.035731,0.6,0.6,NaN,NaN,218.082,45.332629,0.207870,186.340,39.478140,0.211861,175.734,252.586,194.88,242.89,0.0,0.0,0.0,0.0,0.119972,0.058926,0.129138,1.21,0.242,3.223140,0.621024,0.9,0.278976,2.20,0.440,2.522727,0.527717,0.7,0.172283
7590,2024-05-28,Argentina Liga A,20:30,Platense,Obras,1.65,2.19,161.5,1.89,1.85,-2.5,1.85,2.40,0CAxk6RE,0.606061,0.456621,0.529101,0.540541,0.062682,0.4,0.4,NaN,NaN,152.308,31.391479,0.206105,174.620,32.831789,0.188018,123.850,176.952,132.60,144.90,0.0,0.0,0.0,0.0,0.198874,0.015125,0.183016,1.40,0.280,2.321429,0.682687,0.9,0.217313,-0.38,-0.076,-15.657895,0.454332,0.3,-0.154332
7591,2024-05-28,Ásia Wasl,15:00,Kuwait SC,Shahrdari Gorgan,1.76,1.97,172.5,1.85,1.85,-2.5,2.02,2.38,O8YQ2oNF,0.568182,0.507614,0.540541,0.540541,0.075796,0.0,0.0,NaN,NaN,121.482,28.399455,0.233775,NaN,NaN,NaN,0.000,0.000,120.12,2080.00,0.0,1.0,0.0,0.0,0.079621,0.000000,0.115708,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7592,2024-05-28,Bulgária Nbl,13:00,Balkan,Rilski Sportist,2.05,1.70,157.5,1.85,1.85,1.5,1.88,1.58,pYshzTkB,0.487805,0.588235,0.540541,0.540541,0.076040,0.8,1.0,NaN,NaN,139.172,33.561685,0.241153,129.132,21.477287,0.166320,136.770,132.496,159.80,155.70,0.0,0.0,0.0,0.0,0.131993,0.000000,0.122620,1.05,0.210,5.000000,0.000000,0.0,0.000000,-0.74,-0.148,-4.729730,0.669764,0.7,0.030236
7593,2024-05-28,Eua Wnba,21:00,Chicago Sky F,Seattle Storm F,2.40,1.58,162.5,1.80,1.96,2.5,2.01,1.42,dYKF0QT2,0.416667,0.632911,0.555556,0.510204,0.049578,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,158.10,0.0,0.0,0.0,0.0,0.291371,0.060179,0.243261,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7594,2024-05-28,França Lnb,16:05,Lyon-Villeurbanne,Paris,1.69,2.19,163.5,1.80,1.99,-3.5,2.02,2.75,W6NCgV1k,0.591716,0.456621,0.555556,0.502513,0.048337,0.4,0.4,NaN,NaN,148.236,57.550746,0.388237,125.044,31.863083,0.254815,106.902,146.434,120.12,140.00,0.0,0.0,0.0,0.0,0.182244,0.070897,0.216431,1.07,0.214,3.224299,0.000000,0.0,0.000000,-3.25,-0.650,-1.830769,0.000000,0.0,0.000000
7595,2024-05-28,França Pro B,15:00,Rouen,La Rochelle,1.85,1.87,147.5,1.86,1.80,-1.5,1.99,2.02,SxgWJBhC,0.540541,0.534759,0.537634,0.555556,0.075300,0.6,0.0,NaN,NaN,131.916,24.622491,0.186653,112.000,17.064095,0.152358,146.116,125.920,154.50,110.24,0.0,0.0,0.0,0.0,0.007603,0.023184,0.010580,1.92,0.384,2.213542,0.000000,0.0,0.000000,-3.25,-0.650,-1.338462,0.548691,0.5,-0.048691
7596,2024-05-28,Países Baixos Dbl,15:00,Den Bosch,Leiden,1.32,3.20,152.5,1.80,1.86,-7.5,1.92,6.50,jJ0c9b6U,0.757576,0.312500,0.555556,0.537634,0.070076,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,106.20,0.00,0.0,0.0,0.0,0.0,0.588213,0.023184,0.769252,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7597,2024-05-28,Rússia Liga Vtb United,13:30,Unics Kazan,CSKA Moscow,1.80,2.02,165.5,1.84,1.92,-2.5,2.05,2.29,0fyOaO2B,0.555556,0.495050,0.543478,0.520833,0.050605,0.8,1.0,NaN,NaN,167.494,22.953297,0.137040,184.150,27.047828,0.146879,138.142,188.466,160.55,210.24,0.0,0.0,0.0,0.0,0.081447,0.030090,0.078205,0.67,0.134,5.970149,0.772672,0.8,0.027328,1.33,0.266,3.834586,0.500182,0.5,-0.000182
7598,2024-05-28,Taiwan P. League+,08:00,New Taipei Kings,Formosa Dreamers,1.75,1.98,189

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:30,Eua Nba,Dallas Mavericks,Minnesota Timberwolves,209.5,1.84,1.0000,Over
1,20:30,Argentina Liga A,Platense,Obras,161.5,1.89,1.0000,Over
2,15:00,Ásia Wasl,Kuwait SC,Shahrdari Gorgan,172.5,1.85,1.0000,Over
3,16:05,França Lnb,Lyon-Villeurbanne,Paris,163.5,1.80,1.0000,Over
4,15:00,França Pro B,Rouen,La Rochelle,147.5,1.86,1.0000,Over
5,15:00,Países Baixos Dbl,Den Bosch,Leiden,152.5,1.80,1.0000,Over
6,13:30,Rússia Liga Vtb United,Unics Kazan,CSKA Moscow,165.5,1.84,1.0000,Over
7,08:00,Taiwan P. League+,New Taipei Kings,Formosa Dreamers,189.5,1.84,0.9996,Over
8,21:40,Colômbia Lbp,Caribbean Storm Islands,Titanes de Barranquilla,151.5,1.84,0.9999,Over
